ML4HC_Project01_Code:
Part 2


In [4]:
# import packages
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

In [15]:
#install kaggle
! pip install kaggle

In [16]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# make tmp kaggle folder, add json, allow
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/ETH/kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [19]:
# download and unzip dataset
! kaggle datasets download paultimothymooney/chest-xray-pneumonia
! unzip chest-xray-pneumonia.zip

chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  chest-xray-pneumonia.zip
replace chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [20]:
# loading dataset
train = get_training_data('./chest_xray/train/')
test = get_training_data('./chest_xray/test/')
val = get_training_data('./chest_xray/val/')

NameError: name 'get_training_data' is not defined

In [ ]:
# plot examples
plt.figure(figsize = (5,5))
plt.imshow(train[0][0], cmap='gray')
plt.title(labels[train[0][1]])

plt.figure(figsize = (5,5))
plt.imshow(train[-1][0], cmap='gray')
plt.title(labels[train[-1][1]])

In [ ]:
# descriptive stats

In [ ]:
# normalize to [0,1]
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

In [ ]:
# resize data for deep learning
x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [ ]:
# define CNN
net = nn.Sequential(
  nn.Flatten(),
  nn.Linear(784, 10), # 784 inputs, 10 outputs
  nn.LogSoftmax(dim=1)
)


In [22]:
# set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
net = net.to(device)

In [ ]:
# batching
train_loader = torch.utils.data.DataLoader(data_train, batch_size=64)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=64)

In [ ]:
def train_epoch(net, dataloader, device, lr=0.01, optimiser=None, loss_fn = nn.NLLLoss()):
    optimiser = optimiser or torch.optim.Adam(net.parameters(), lr=lr)
    # Ensure the network is in training mode
    net.train()
    total_loss, acc, count = 0,0,0
    for features,labels in dataloader:
        # Ensure features/images and labels are on the correct device
        features, labels = features.to(device), labels.to(device)
        # Predict outputs for features and compute the loss between predictions
        # and labels
        out = net(features)
        loss = loss_fn(out, labels)
        total_loss += loss

        # Zero gradients from earlier computations
        optimiser.zero_grad()
        # Compute gradients for weights
        loss.backward()
        # Do optimiser step
        optimiser.step()

        # Get predicted classes as the entry with the highest probability
        _, predicted = torch.max(out, 1)
        # Compute accuracy
        acc += (predicted == labels).sum().cpu()
        count += len(labels)
    return total_loss.item()/count, acc.item()/count

train_epoch(net, train_loader, device)

In [ ]:
def validate_epoch(net, dataloader, device, loss_fn=nn.NLLLoss()):
    # Ensure network is in evaluation mode
    net.eval()
    count,acc,loss = 0,0,0
    # We use torch.no_grad() to remove gradient computations
    # This is not necessary, but can be much faster
    with torch.no_grad():
        for features,labels in dataloader:
            # Ensure features and labels are on the correct device
            features = features.to(device)
            labels = labels.to(device)
            # Make predictions
            out = net(features)
            # Compute loss and accuracy of the model
            loss += loss_fn(out,labels)
            pred = torch.max(out,1)[1]
            acc += (pred==labels).sum()
            count += len(labels)
    return loss.item()/count, acc.item()/count

validate_epoch(net, test_loader, device)

Overfitting?

In [ ]:
#access overfitting
def train(net, train_loader, test_loader, device=torch.device('cpu'), optimiser=None, lr=0.01, epochs=10, loss_fn=nn.NLLLoss()):
    optimiser = optimiser or torch.optim.Adam(net.parameters(),lr=lr)
    res = { 'train_loss' : [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    for ep in range(epochs):
        tl,ta = train_epoch(
            net=net,
            dataloader=train_loader,
            device=device,
            optimiser=optimiser, lr=lr, loss_fn=loss_fn)
        vl,va = validate_epoch(
            net=net,
            dataloader=test_loader,
            device=device,
            loss_fn=loss_fn)
        print(f"Epoch {ep:2}, Train acc={ta:.3f}, Val acc={va:.3f}, Train loss={tl:.3f}, Val loss={vl:.3f}")
        res['train_loss'].append(tl)
        res['train_acc'].append(ta)
        res['val_loss'].append(vl)
        res['val_acc'].append(va)
    return res

# Re-initialize the network to start from scratch
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 10),
    nn.LogSoftmax(dim=1)
).to(device)

# This will take a minute or two
hist = train(
    net=net,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    epochs=10
)



In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(hist['train_acc'], label='Training acc')
plt.plot(hist['val_acc'], label='Validation acc')
plt.legend()
plt.subplot(122)
plt.plot(hist['train_loss'], label='Training loss')
plt.plot(hist['val_loss'], label='Validation loss')
plt.legend()

Integrated Gradients

Grad-CAM

https://github.com/jacobgil/pytorch-grad-cam

Data Randomization Test